In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

# %pip install numpy-financial
import numpy_financial as npf

pd.options.display.float_format = '{:,.2f}'.format
# pd.set_option("max_columns", None) #Showing only two columns
# pd.set_option("max_rows", None)


# open C:\Users\Admin\Desktop\Portfolio\Housing market\ Real Estate_realtor.xlsx
df = pd.read_excel(r'C:\Users\Admin\Desktop\Portfolio\Housing market\Real Estate_realtor.xlsx')

def add_region_column(df):
    conditions = [
        (df['Latitud'] > 49.8865426) & (df['Longitud'] < -97.14173217),  # Noroeste
        (df['Latitud'] > 49.8865426) & (df['Longitud'] > -97.14173217),  # Noreste
        (df['Latitud'] < 49.8865426) & (df['Longitud'] < -97.14173217),  # Suroeste
        (df['Latitud'] < 49.8865426) & (df['Longitud'] > -97.14173217)   # Sureste
    ]
    choices = ['Noroeste', 'Noreste', 'Suroeste', 'Sureste']
    df['region'] = np.select(conditions, choices, default='Otro')
    return df
df = add_region_column(df)

df = df.rename(columns={'smalllistingcardiconcon': 'bedrooms',
                        'smalllistingcardiconcon1': 'bathrooms',
                        'Precio': 'price',
                        'Título': 'location'})

# replace $ and , in Price column
df['price'] = df['price'].str.replace('$','').str.replace(',','').astype(float)
#crea una nueva columna donde separas el texto de la columna location en dos columnas diferentes
df[['block','city','province']] = df['location'].str.split(',',expand=True)


def find_last_number_position(text):
    last_number_position = -1
    for i, char in enumerate(text):
        if char.isnumeric():
            last_number_position = i
    if last_number_position != -1:
        return text[:last_number_position+1]
    return None
df['number'] = df['block'].apply(find_last_number_position)


def remove_numbers(text):
    return re.sub(r'\d', '', text)
df['block'] = df['block'].apply(remove_numbers)
df['block'] = df['block'].str.strip()
df['download_date'] = datetime.now().date()
# df['price'] = df['price']/1000
df['area'] = df['Título_URL'].str.split('winnipeg-', n=1).str[1]
import numpy_financial as npf

def calculate_monthly_payment(price, mortgage_years, down_payment, interest_rate):
    interest_rate_monthly = (interest_rate / 100) / 12
    mortgage_months = mortgage_years * 12
    mortgage_amount = price - (price * down_payment)
    monthly_payment = npf.pmt(interest_rate_monthly, mortgage_months, -mortgage_amount)
    
    return monthly_payment

mortgage_years = 25  # Plazo del préstamo hipotecario en años
down_payment = 0.2  # Porcentaje de pago inicial
interest_rate = 7.0  # Tasa de interés anual
df['normal_payment'] = calculate_monthly_payment(df['price'], mortgage_years, down_payment, interest_rate).round(2)

mortgage_years = 3  # Plazo del préstamo hipotecario en años
down_payment = 0.2  # Porcentaje de pago inicial
interest_rate = 7.0  # Tasa de interés anual
df['stress_payment'] = calculate_monthly_payment(df['price'], mortgage_years, down_payment, interest_rate).round(2)
# df = df.drop(columns=['Imagen', 'Título_URL'])


def add_budget_column(df):
    conditions = [
        (df['normal_payment'] >= 1100),
        (df['normal_payment'] < 1100)
    ]
    choices = ['under', 'below']
    df['budget'] = np.select(conditions, choices, default='Otro')
    return df
df = add_budget_column(df)

display(df)
df.to_excel(r'C:\Users\Admin\Desktop\Portfolio\Housing market\Real Estate_realtor_clean.xlsx', index = False, header=True)

,location,Título_URL,Imagen,price,bedrooms,bathrooms,Etiqueta,Latitud,Longitud,region,block,city,province,number,download_date,area,normal_payment,stress_payment,budget
0,"183 Nordstrom Drive, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26154463/18...,https://cdn.realtor.ca/listings/TS638325441657...,"599,900.00",5.00,4.00,15 min ago,49.83,-97.05,Sureste,Nordstrom Drive,Winnipeg,Manitoba,183,2023-10-12,island-lakes,"3,391.97","14,818.54",under
1,"344 Riverton Avenue, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26154462/34...,https://cdn.realtor.ca/listings/TS638325441656...,"139,900.00",2.00,2.00,15 min ago,49.91,-97.12,Noreste,Riverton Avenue,Winnipeg,Manitoba,344,2023-10-12,elmwood,791.03,"3,455.76",below
2,"94 Bridgeland Drive S, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26154460/94...,https://cdn.realtor.ca/listings/TS638325441615...,"739,900.00",2.00,2.00,15 min ago,49.80,-97.18,Suroeste,Bridgeland Drive S,Winnipeg,Manitoba,94,2023-10-12,bridgwater-lakes,"4,183.57","18,276.77",under
3,"6 409 Oakdale Drive, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26154459/6-...,https://cdn.realtor.ca/listings/TS638325441280...,"134,900.00",1.00,1.00,15 min ago,49.86,-97.27,Suroeste,Oakdale Drive,Winnipeg,Manitoba,6 409,2023-10-12,charleswood,762.76,"3,332.26",below
4,"57 Blackmore Avenue, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26154450/57...,https://cdn.realtor.ca/listings/TS638325441591...,"284,900.00",2.00,1.00,17 min ago,49.82,-97.14,Sureste,Blackmore Avenue,Winnipeg,Manitoba,57,2023-10-12,bright-oaks,"1,610.89","7,037.51",under
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,"12 1424 Dakota Street, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26092895/12...,https://cdn.realtor.ca/listings/TS638310068093...,"199,900.00",2.00,1.00,NaN,0.00,0.00,Sureste,Dakota Street,Winnipeg,Manitoba,12 1424,2023-10-12,river-park-south,"1,130.28","4,937.87",under
592,"344 Turnbull Drive, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26092894/34...,https://cdn.realtor.ca/listings/TS638310032458...,"798,900.00",3.00,3.00,NaN,49.75,-97.14,Sureste,Turnbull Drive,Winnipeg,Manitoba,344,2023-10-12,st-norbert,"4,517.17","19,734.17",under
593,"1328 Arlington Street, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26092893/13...,https://cdn.realtor.ca/listings/TS638310068086...,"169,900.00",4.00,2.00,NaN,49.91,-97.16,Noroeste,Arlington Street,Winnipeg,Manitoba,1328,2023-10-12,north-end,960.65,"4,196.82",below
594,"798 Harbison Avenue E, Winnipeg, Manitoba",https://www.realtor.ca/real-estate/26092339/79...,https://cdn.realtor.ca/listings/TS638310032453...,"184,900.00",2.00,1.00,NaN,49.91,-97.09,Noreste,Harbison Avenue E,Winnipeg,Manitoba,798,2023-10-12,elmwood,"1,045.47","4,567.34",below


In [8]:
variables = df[['price', 'bedrooms', 'bathrooms','Latitud',	'Longitud']]
correlation_matrix = variables.corr()
pd.options.display.float_format = '{:,.4f}'.format
display(correlation_matrix)

,price,bedrooms,bathrooms,Latitud,Longitud
price,1.0000,0.3850,0.6279,-0.1262,0.1255
bedrooms,0.3850,1.0000,0.6723,0.0960,-0.0963
bathrooms,0.6279,0.6723,1.0000,0.0126,-0.0133
Latitud,-0.1262,0.0960,0.0126,1.0000,-1.0000
Longitud,0.1255,-0.0963,-0.0133,-1.0000,1.0000


In [ ]:
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter
# import time

# # Función para obtener la latitud y longitud de una dirección con reintentos y tiempo de espera
# def get_lat_lon(location):
#     try:
#         geolocator = Nominatim(user_agent="GetLoc")
#         geocode = RateLimiter(geolocator.geocode, min_delay_seconds=4, max_retries=3)
#         location = geocode(location)
#         if location:
#             return location.latitude, location.longitude
#         else:
#             return None, None
#     except Exception as e:
#         print(f"Error: {str(e)}")
#         return None, None

# # Aplica la función a la columna "location"
# df['Latitud'], df['Longitud'] = zip(*df['location'].apply(get_lat_lon))

# # Muestra el DataFrame resultante
# df


In [114]:
import pandas as pd
import plotly.graph_objects as go


# Eliminar filas con valores NaN en latitud o longitud
df = df.dropna(subset=['Latitud', 'Longitud'])

# Crear un gráfico de dispersión geográfica en Plotly
fig = go.Figure(go.Scattermapbox(
    lat=df['Latitud'],
    lon=df['Longitud'],
    mode='markers',
    marker=dict(size=10, color='red', opacity=0.7),
    text=df[[ 'price', 'location', 'bedrooms', 'bathrooms']],
))

fig.update_layout(
    mapbox=dict(
        style='open-street-map',  # Utiliza un estilo similar a OpenStreetMap
        center=dict(lon=-97.1384, lat=49.8951),
        zoom=11
    ),
    width=1200,  # Ancho del gráfico
    height=1000  # Alto del gráfico
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
# Mostrar el gráfico interactivo
fig.show()

In [9]:
# from sklearn.cluster import KMeans

# # Selecciona las columnas que se utilizarán para el agrupamiento
# features = df[['price', 'bedrooms', 'bathrooms', 'Latitud', 'Longitud']]
# features = features.dropna()

# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='mean')  # Puedes elegir la estrategia que desees (media, mediana, etc.)

# features = imputer.fit_transform(features)

# # Especifica el número de clusters que deseas
# n_clusters = 3  # Puedes ajustar este valor según tus necesidades

# # Crea un modelo de K-Means
# kmeans = KMeans(n_clusters=n_clusters)

# # Ajusta el modelo a tus datos y obtén las etiquetas de cluster
# df['class'] = kmeans.fit_predict(features)


In [13]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Crear un DataFrame con datos de ventas de viviendas
data = {
    'Size (sq. ft.)': [1500, 1800, 1200, 2000, 1600],
    'Bedrooms': [3, 4, 2, 4, 3],
    'Location': ['Norte', 'Sur', 'Sur', 'Este', 'Oeste'],
    'Price ($)': [250000, 350000, 200000, 380000, 280000]
}

df = pd.DataFrame(data)

# Crear variables ficticias (dummy variables) para la ubicación geográfica
df = pd.get_dummies(df, columns=['Location'], drop_first=True)

# Aplicar un modelo de regresión lineal para calcular el precio hedónico
X = df[['Size (sq. ft.)', 'Bedrooms', 'Location_Norte', 'Location_Sur', 'Location_Este', 'Location_Oeste']]
y = df['Price ($)']

model = LinearRegression()
model.fit(X, y)

# Supongamos que queremos calcular el precio de una vivienda con los siguientes atributos
new_data = {
    'Size (sq. ft.)': [1700],
    'Bedrooms': [3],
    'Location_Norte': [1]  # Ubicado en el Norte
}

new_df = pd.DataFrame(new_data)

# Realizar una predicción del precio hedónico
predicted_price = model.predict(new_df)

print("El precio hedónico estimado es:", predicted_price[0])


KeyError: "['Location_Este'] not in index"